# 얼굴을 인식하여 캐릭터 씌우기

### Face Detection vs. Face Recognition

In [4]:
import numpy as np

import cv2
import mediapipe as mp

video_path = './Data/face_video.mp4'

In [18]:
# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils # 얼골의 특징을 그리기 위한 drawing_utils 모듈 사용

# 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

# 이미지 불러오기

# 투명도 적용하기
def overlay(image, x, y, w, h, overlay_img):
    alpha = overlay_img[:, :, 3] # BGRA
    mask_img = alpha / 255 # (1: 불투명, 0 : 완전투명)
    
    for c in range(0, 3):
        image[y-h:y+h, x-w:x+w, c] = (overlay_img[:, :, c] * mask_img) + (image[y-h:y+h, x-w:x+w, c] * (1-mask_img))


with mp_face_detection.FaceDetection(model_selection = 0,
                                    min_detection_confidence = 0.7) as face_detection:
    # model_selection = (0 or 1) 가까운 거리 0, 먼 거리 1
    # min_detection_confidence = 어느정도 신뢰도로 얼굴로 판단을 내리고 있는지
    
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(frame)

        # Draw the face detection annotations on the image.
        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝, 입 중심, 오른쪽 퀴, 왼쪽 귀
            for detection in results.detections:
#                 mp_drawing.draw_detection(frame, detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]
                
                h, w, _ = frame.shape
                right_eye = (int(right_eye.x * w)-20, int(right_eye.y * h)-100)
                left_eye = (int(left_eye.x * w), int(left_eye.y * h))
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
#                 # 양 눈에 동그라미 그리기
#                 cv2.circle(frame, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA)
#                 cv2.circle(frame, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA)
#                 # 코에 동그라미 그리기
#                 cv2. circle(frame, nose_tip, 75, (0, 255, 255), 10, cv2.LINE_AA)

#                 # 각 특징에 이미지 그리기
#                 image[right_eye[1]-50:right_eye[1]+50,
#                       right_eye[0]-50:right_eye[1]+50] = image_right_eye
#                 image[left_eye[1]-50:left_eye[1]+50,
#                       left_eye[0]-50:left_eye[1]+50] = image_left_eye
#                 image[nose_tip[1]-50:nose_tip[1]+50,
#                       nose_tip[0]-150:nose_tip[1]+150] = image_nose

                # ovaray image 적용
                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose_tip, 50, 50, image_nose)

        
        cv2.imshow('MediaPipe Face Detection', cv2.resize(frame, None, fx = 0.5, fy = 0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

### 학습 자료
> https://opencv-python.readthedocs.io/en/latest/